In [1]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt 
from build_discriminator import build_discriminator
from build_generator import build_generator
import numpy as np 

In [12]:
def train(models,x_train,params):
    generator, discriminator, adversarial = models

    batch_size,latent_size,train_steps,model_name = params 

    save_interval = 500 #500단계 마다 생성기 이미지 저장 

    noise_input = np.random.uniform(-1,1,size=[16,latent_size]) #훈련 동안 생성기 변화 확인 

    train_size = x_train.shape[0]

    for i in range(train_steps):
        #진짜 데이터 
        rand_indexes = np.random.randint(0,train_size,size=batch_size)
        real_iamges = x_train[rand_indexes]

        #가짜 데이터 
        noise = np.random.uniform(-1,1,size=[batch_size,latent_size])
        fake_images = generator.predict(noise)

        #진짜 데이터 + 가짜 데이터 = 훈련 데이터의 1 배치 

        x = np.concatenate((real_iamges,fake_images))

        y = np.ones([2*batch_size,1]) # 정답 레이블 
        y[batch_size:,:] = 0 #가짜 정답 레이블 

        #판별기, 네트워크 훈련, 손실 정확도 기록 
        loss,acc = discriminator.train_on_batch(x,y)
        log = "%d: [discriminator loss : %f, acc : %f]" % (i,loss,acc)


        noise = np.random.uniform(-1,1,size=[batch_size,latent_size])
        y = np.ones([batch_size,1])

        loss,acc = adversarial.train_on_batch(noise,y)
        log = "%s [adversarial loss :%f, acc :%f" % (log,loss,acc)

        

        if (i+1) % save_interval ==0:
            print(log)
            plot_images(generator,
                        noise_input=noise_input)
                        
def plot_images(generator,noise_input,show):
    a = generator.predict(noise_input)
    plt.figure(figsize=(10,10))
    for i in range(16):
        plt.subplot(5,5,i+1)
        plt.imshow(a[i], cmap="gray")
    plt.show()
                        

In [13]:
# def build_and_train_models():
(x_train,_),(_,_) = mnist.load_data()

# CNN을 위한 데이터 형상을 28,28,1로 조정하고 정규화 
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1,image_size,image_size,1])
x_train = x_train.astype('float32')/255

model_name = 'dcgan_mnist'

latent_size = 100 #z벡터 차원 수 
batch_size = 64 
train_steps = 40000
lr = 2e-4
decay = 6e-8
input_shape = (image_size,image_size,1)

#식별자 모델 구성 
inputs = Input(shape=input_shape,name='discriminator_inputs')
discriminator = build_discriminator(inputs)
optimizer = RMSprop(learning_rate=lr, decay =decay)
discriminator.compile(loss = 'binary_crossentropy',
                        optimizer = optimizer,
                        metrics = ['accuracy'])
#생성자 모델 구성 

input_shape = (latent_size,)
inputs = Input(shape = input_shape, name = 'z_input')
generator = build_generator(inputs,image_size)

#적대적 모델 구성 
optimizer = RMSprop(learning_rate=lr*0.5, decay = decay *0.5)
#적대적 모델을 훈련하는 동안 판별기 가중치 고정 
discriminator.trainable =False
# adversial = generator + discriminator
adversarial = Model(inputs,
                        discriminator(generator(inputs)),
                        name = model_name)
adversarial.compile(loss = 'binary_crossentropy',
                         optimizer = optimizer,
                         metrics = ['accuracy'])

#판별기와 적대적 네트워크를 훈련 
models = (generator, discriminator, adversarial)
params = (batch_size,latent_size,train_steps,model_name)
train(models,x_train,params)


499: [discriminator loss : 0.796239, acc : 0.570312] [adversarial loss :2.516796, acc :0.000000
999: [discriminator loss : 0.647936, acc : 0.601562] [adversarial loss :2.017572, acc :0.000000
1499: [discriminator loss : 0.506160, acc : 0.773438] [adversarial loss :1.252927, acc :0.109375


KeyboardInterrupt: 